In [1]:
# CELL 1: Install packages
!pip install -q \
    transformers \
    torch \
    faiss-gpu \
    sentence-transformers \
    beautifulsoup4 \
    tqdm \
    unidecode \
    pandas \
    numpy \
    scikit-learn \
    gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 27.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.4/136.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip -q install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.2 MB/s eta 0:00:00:00:0100:01


In [3]:
# CELL 2: Imports & Config
import os, re, json, random, math
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
from tqdm import tqdm
from typing import List, Dict
import faiss
import pickle
import torch

from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import pipeline, AutoTokenizer

# === CẤU HÌNH ===
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")

# Model
BI_ENCODER_MODEL = "bkai-foundation-models/vietnamese-bi-encoder"
CROSS_ENCODER_MODEL = "namdp-ptit/ViRanker"
LLM_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"  # hoặc Qwen2.5-1.5B nếu GPU yếu

# Crawl
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
TIMEOUT = 20
SLEEP = 0.6

# Chunking
MAX_TOKENS = 350
MIN_CHARS = 100
OVERLAP_TOKENS = 50

Device: cuda


In [4]:
# ==========================================================
# CELL 4 (SỬA LỖI): Semantic Chunking + Overlap
# ==========================================================
# --- Thêm các biến và imports còn thiếu ---
# Giả sử các giá trị này từ các cell trước

# Tên file CSV đầu vào (đã sửa đường dẫn)
INPUT_CSV_PATH = "/kaggle/input/seg-5k-crawl/crawled_content_from_csv.csv"

# --- Code chunking của bạn (giữ nguyên) ---
# Tải bi_encoder (mặc dù không dùng trong logic chunking)
# print(f"Loading Bi-Encoder: {BI_ENCODER_MODEL}...")
# bi_encoder = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)
# print("Bi-Encoder loaded.")

def split_sentences(text: str) -> List[str]:
    if not isinstance(text, str):
        return [] # Xử lý trường hợp content_text là NaN
    sentences = re.split(r'(?<=[.!?])\s*\n?\s*|(?<=[.!?])\s+', text)
    return [s.strip() for s in sentences if s.strip() and len(s) > 10]

def merge_small(sentences: List[str], min_chars=100) -> List[str]:
    merged, current, chars = [], [], 0
    for s in sentences:
        if chars + len(s) < min_chars and current:
            current.append(s)
            chars += len(s) + 1
        else:
            if current:
                merged.append(' '.join(current))
            current, chars = [s], len(s)
    if current and len(' '.join(current)) >= 50:
        merged.append(' '.join(current))
    return merged

def semantic_chunking(text: str) -> List[Dict]:
    sentences = merge_small(split_sentences(text), min_chars=80)
    chunks = []
    i = 0
    while i < len(sentences):
        chunk, tokens = [], 0
        j = i
        while j < len(sentences) and tokens + len(sentences[j].split()) <= MAX_TOKENS:
            chunk.append(sentences[j])
            tokens += len(sentences[j].split())
            j += 1

        chunk_text = ' '.join(chunk)
        if len(chunk_text) >= MIN_CHARS:
            # Overlap nếu cần
            # (Lưu ý: logic overlap này sẽ thêm các câu *trước đó* vào *đầu* chunk hiện tại)
            if chunks and OVERLAP_TOKENS > 0 and i >= OVERLAP_TOKENS:
                # Tìm index câu thực tế để bắt đầu overlap
                # Logic này hơi phức tạp, cần xem lại kỹ
                # Giả sử 'OVERLAP_TOKENS' là số *câu* (sentences) chứ không phải *tokens*
                overlap_sents_count = 3 # Ví dụ: overlap 3 câu
                if i >= overlap_sents_count:
                    overlap_sents = sentences[i - overlap_sents_count : i]
                    chunk_text = ' '.join(overlap_sents) + ' ' + chunk_text

            chunks.append({'text': chunk_text})
        
        # Logic bước nhảy:
        # Nếu chunk quá nhỏ (j <= i+2), chỉ nhảy 1 câu để thử gộp
        # Nếu chunk lớn, nhảy đến cuối chunk (j)
        i = j if j > i + 2 else i + 1 
    return chunks

# --- Áp dụng chunking (ĐÃ SỬA LỖI) ---
print("Đang đọc file CSV...")
try:
    df = pd.read_csv(INPUT_CSV_PATH)
    
    print("Đang chunking...")
    all_chunks = []
    
    # Sử dụng tqdm(df.iterrows(), total=df.shape[0])
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        chunks = semantic_chunking(row['content_text']) # Đọc từ 'content_text'
        for c in chunks:
            all_chunks.append({
                'text': c['text'],
                # === SỬA LỖI Ở ĐÂY ===
                # Sử dụng tên cột viết thường từ file CSV
                'title': row['title'],
                'url': row['url'],
                'chuyen_khoa': row['chuyen_khoa']
                # =====================
            })

    print(f"Đã tạo {len(all_chunks):,} chunk chất lượng.")
    
    # In thử 1 chunk để xem kết quả
    if all_chunks:
        print("\nChunk đầu tiên làm ví dụ:")
        print(all_chunks[0])

except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {INPUT_CSV_PATH}")
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

Đang đọc file CSV...
Đang chunking...


100%|██████████| 4945/4945 [00:03<00:00, 1322.89it/s]

Đã tạo 22,657 chunk chất lượng.

Chunk đầu tiên làm ví dụ:
{'text': 'Bệnh viêm phúc mạc ở trẻ em\\n\\nBài được viết bởi Bác sĩ Lê Văn Bình - Khoa Hồi sức tích cực - Bệnh viện Đa khoa Quốc tế Vinmec Times City. \\n\\nViêm phúc mạc là tình trạng đỏ và sưng (viêm) mô lót bụng hoặc vùng bụng của bạn. Mô này được gọi là phúc mạc. Đây có thể là một căn bệnh nguy hiểm chết người nếu không được điều trị kịp thời và đúng cách. Bệnh không chỉ xảy ra ở người trưởng thành mà còn có thể bắt gặp ở trẻ nhỏ. Tổng quan về bệnh viêm phúc mạc\\n\\nViêm phúc mạc là tình trạng viêm của lá thành - lá tạng khoang màng bụng do nguyên nhân nhiễm trùng hoặc không nhiễm trùng. \\n\\nViêm phúc mạc tiên phát. \\n\\nViêm phúc mạc thứ phát. \\n\\nViêm phúc mạc kết hợp (bệnh nhân chạy thận nhân tạo). \\n\\nNguyên nhân gây viêm phúc mạc có thể do:\\n\\nNhiễm khuẩn: Thủng ống tiêu hóa , chấn thương, tiên phát. \\n\\nKhông nhiễm khuẩn: Phẫu thuật vô trùng ổ bụng, dò dịch vô trùng vào ổ bụng, bệnh hiếm gặp,. Biểu hiện lâ

In [9]:
# ==========================================================
# Bước 3: Tải Model và Dữ liệu (chỉ tải 1 lần)
# ==========================================================
print(f"Đang tải model trên {DEVICE}...")
bi_encoder = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)
cross_encoder = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
generator = pipeline(
    "text-generation",
    model=LLM_MODEL,
    tokenizer=llm_tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map=DEVICE,
    pad_token_id=llm_tokenizer.eos_token_id,
    eos_token_id=llm_tokenizer.eos_token_id,
    max_new_tokens=512
    #"load_in_8bit": True
     )
  

Đang tải model trên cuda...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [10]:
# CELL 5: Build FAISS
print("Tạo embedding...")
embeddings = bi_encoder.encode(
    [c['text'] for c in all_chunks],
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype('float32')

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

# Lưu
faiss.write_index(index, "vinmec_faiss_token_chunk.index")
with open("vinmec_chunks.pkl", "wb") as f:
    pickle.dump(all_chunks, f)

print(f"FAISS + {len(all_chunks)} chunks đã lưu!")

Tạo embedding...


Batches:   0%|          | 0/355 [00:00<?, ?it/s]

FAISS + 22657 chunks đã lưu!


In [14]:
# CELL 6: RAG System
cross_encoder = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)

# Load FAISS + chunks
index = faiss.read_index("/kaggle/input/seg-5k-crawl/vinmec_faiss_token_chunk.index")
with open("/kaggle/input/seg-5k-crawl/vinmec_chunks.pkl", "rb") as f:
    chunks = pickle.load(f)

# LLM
generator = pipeline(
    "text-generation",
    model=LLM_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    max_new_tokens=300,
    temperature=0.3,
    do_sample=True
)

def retrieve(query: str, k=30) -> List[Dict]:
    q_emb = bi_encoder.encode([query], normalize_embeddings=True)
    D, I = index.search(q_emb, k)
    return [chunks[i] for i in I[0]]

def rerank(query: str, docs: List[Dict], topk=5) -> List[Dict]:
    pairs = [[query, d['text']] for d in docs]
    scores = cross_encoder.predict(pairs)
    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [r[0] for r in ranked[:topk]]

def rag_answer(query: str) -> str:
    docs = retrieve(query, k=30)
    top_docs = rerank(query, docs, topk=5)

    context = "\n\n".join([
        f"[{d['title']}]: {d['text'][:1200]}"
        for d in top_docs
    ])

    prompt = f"""Bạn là bác sĩ chuyên khoa Nhi và Phụ nữ.
Dựa vào thông tin sau, trả lời ngắn gọn, dễ hiểu:

{context}

Câu hỏi: {query}
Trả lời:"""

    output = generator(prompt)[0]['generated_text']
    answer = output.split("Trả lời:")[-1].strip()
    return answer, top_docs

In [15]:
bi_encoder = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)

In [16]:
# CELL 7: Test
query = "Ung thư tử cung"
answer, sources = rag_answer(query)

print("CÂU HỎI:", query)
print("\nTRẢ LỜI:")
print(answer)
print("\nNGUỒN:")
for s in sources:
    print(f"- {s['title']} ({s['url']})")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CÂU HỎI: Ung thư tử cung

TRẢ LỜI:
Ung thư cổ tử cung là một loại ung thư xảy ra trong các tế bào của cổ tử cung. Nguyên nhân chính gây ra bệnh này là virus papilloma ở người (HPV), đây là bệnh lây truyền qua đường tình dục. Người bệnh có thể giảm nguy cơ phát triển ung thư cổ tử cung bằng cách khám sàng lọc và tiêm vắc-xin chống nhiễm trùng HPV. Ung thư cổ tử cung thường phát triển chậm, vì vậy, người bệnh có thời gian để phát hiện và điều trị trước khi nó gây ra các vấn đề nghiêm trọng hơn. 

Phương pháp điều trị thông thường có thể kể đến là hóa trị và xạ trị. Nếu bệnh nhân kiểm tra dương tính với đột biến gen BRCA, phẫu thuật cắt tử cung có thể không cần thiết. Thay vào đó, các bác sĩ có thể đề xuất loại bỏ buồng trứng và ống dẫn trứng vì bệnh nhân đột biến gen này có nguy cơ cao mắc phải ung thư buồng trứng, ung thư vú. 

Bệnh lạc nội mạc tử cung là một dạng rối loạn, là tình trạng mô lót bên trong niêm mạc tử cung phát triển không bình thường, lan rộng ra các khu vực bên ngoài. T